# Effectiveness and computational efficiency of absorbing boundary conditions for full-waveform inversion

## Authors: Daiane Iglesia Dolci, Felipe A. G. Silva, Pedro S. Peixoto and Ernani V. Volpe

## Department of Mechanical Engineering of Polytechnic School, University of São Paulo

## Department of Applied Mathematics, Institute of Mathematics and Statistics, University of São Paulo

## Contacts: dolci@usp.br, felipe.augusto.guedes@usp.br, pedrosp@ime.usp.br, ernvolpe@usp.br

**Important Informations:** These codes are part of the Project Software Technologies for Modeling and Inversion (STMI) at RCGI in the  University of Sao Paulo.

This notebook has the main instructions to carry out the analyses of ABCs methods in the forward and adjoint solvers. Also, to execute a forward solver to store the receiver's data related to the Marmousi and 2D SEG/EAGE velocity models. The FWI solver may be executed from this current notebook.

In [1]:
# Importing Python Packges

import os                                                                       
import multiprocessing as mp
import numpy as np
import argparse

In [2]:
def run_python(process):
    
    os.system('python3 {}'.format(process)) 

Velocity models that can be used in the ABCs analyses:

<span style='color:Blue'>
    
>model = 'Marmousi' 

>model = 'Salt' #2D SEG/EAGE 

>model = 'Circle' 

>model = 'HorizontalLayers' 
</span>

Case to be executed:

<span style='color:Red'>
    
>case = 'fwd_reference'

>case = 'bwd_reference'

>case = 'fwd_test'

>case = 'bwd_test'

>case = 'fwi'

>case = 'true_rec'
</span>

The cases <span style='color:Red'> fwd_reference</span> and <span style='color:Red'>adjoint_reference</span> execute respectively the forward and adjoint solvers to store the pressure field computed in the reference domain.

The case  <span style='color:Red'>fwd_test</span> and <span style='color:Red'>bwd_test</span> execute the respectively forward and adjoint tests by varying the extension domain width and Ricker wavelet peak of frequency.

The case <span style='color:Red'>true_rec</span> executes forward solver to obtain the receivers'data related to the Marmousi and 2D SEG/EAGE vel. models.

The case <span style='color:Red'>fwi</span> executes a FWI solver.

In [3]:
# Setting the case and the velocity model.

case  = 'fwd_reference'
model = 'HorizontalLayers'

In [4]:
if(case=='fwi' or case=='true_rec'):
    
    freq = [15]

else:

    freq = [5, 10, 15, 20]

In [5]:
if(model=='Marmousi'):
    
    if(case=='fwd_reference' or case=='bwd_reference' or case=='true_rec'):
      
        extension = [7500]       # extension lenght (meters)
        methods   = ['damping']  # ABC method
        
    elif(case=='bwd_test' or case=='fwd_test'):
        
        extension = [900, 450, 200, 100] 
        methods   = ['damping' , 'habc-a1', 'Higdon', 'pml', 'cpml']
        print(case)
    
    elif(case=='fwi'):
        
        extension = [400]
        methods   = ['damping']

elif(model=='Salt'):
    
    if(case=='fwd_reference' or case=='bwd_reference' or case=='true_rec'):
        
        extension = [8400]
        methods   = ['damping']

    elif(case=='bwd_test' or case=='fwd_test'):
        
        extension = [1440, 720, 360, 180, 90]
        methods   = ['damping' , 'habc-a1', 'Higdon', 'pml', 'cpml']
    
    elif(case=='fwi'):
        
        extension = [600]
        methods   = ['damping']

elif(model=='Circle' or model=='HorizontalLayers'):
    
    if(case=='fwd_reference' or case=='bwd_reference'):
        
        extension = [2000]
        methods   = ['damping']

    elif(case=='bwd_test' or case=='fwd_test' or case=='true_rec'):
    
        extension = [180, 150, 120, 90, 50] 
        methods   = ['damping' , 'habc-a1', 'Higdon', 'pml', 'cpml']

Execute the case previously chosen.

The ABCs tests varying the peak of frequencies run in parallel.

In [6]:
for i in methods:
    
    for k in extension:
    
        processes = []
        
        for j in freq:

            if(case=='bwd_reference' or case=='bwd_test'):
                
                processes.append("adjoint.py --extension " 
                                + str(k) 
                                + " --freq " 
                                + str(j) 
                                + " --method " 
                                + str(i)
                                + " --model " 
                                + model
                                )     

            elif(case=='fwd_reference' or case=='fwd_test' or case=='true_rec'):
                
                processes.append("forward.py --extension " 
                                + str(k) 
                                + " --freq " 
                                + str(j) 
                                + " --method " 
                                + str(i)
                                + " --model " 
                                + model
                                )                   

            elif(case=='fwi'): #either Marmousi or 2D salt
                 
                processes.append("fwi.py --extension " 
                                + str(k) 
                                + " --freq " 
                                + str(j) 
                                + " --method " 
                                + str(i)
                                + " --model " 
                                + model
                                )  
            
        pool = mp.Pool(processes=len(freq))                                              
        pool.map(run_python, processes)  
        pool.close()
        pool.join()

KeyboardInterrupt: 